In [2]:
import cv2
import numpy as np
import copy
from skimage.measure import label, regionprops
from sklearn.mixture import GaussianMixture as GMM
# import parameters as params
import os
import functools

from components import *
from skimage import measure
import matplotlib.pyplot as plt
from scipy.ndimage import binary_fill_holes
import mahotas
from scipy.signal import find_peaks
from fourier import *
from utility import *

ModuleNotFoundError: No module named 'cv2'

In [72]:
def read_input(path):
    img = cv2.imread(path)
    img = img[:,:,0]
    print('img shape: ',img.shape)
    return img

def denoised(img):
    denoised_img = cv2.fastNlMeansDenoising(img,None,30,7,30)
#     cv2.imwrite(os.path.join(output_path, 'denoised.jpg'),denoised_img)
    return denoised_img

def binarized(img, threshold = -1):
    th, img_th = cv2.threshold(img, 0, 255, cv2.THRESH_BINARY+cv2.THRESH_OTSU)
    print(' OT threshold: ',th)
    #print('threshold using: ',th+10)
    th, img_th = cv2.threshold(img, th+10, 255 , cv2.THRESH_BINARY)
    #cv2.imwrite(os.path.join(output_path,'binarized.jpg'),img_th)
    #print('threshold using: ',th)
    return img_th, th

def pre_processing(path):
    img = read_input(path)
    img_denoised = denoised(img)
    img_bin, th = binarized(img_denoised)
    return img, img_denoised, img_bin, th


In [1]:
layout_path = "1.png"  # Replace with your image path
layout, layout_denoised, layout_bin, th = pre_processing(layout_path)
blobs_layout = connected_components(layout_bin)

plt.imshow(layout_bin, cmap='gray', vmin=0, vmax=1)
SEM_path = layout_path  # Replace with your image path
SEM, SEM_denoised, SEM_bin, th = pre_processing(SEM_path)
blobs_SEM = connected_components(SEM_bin)

NameError: name 'pre_processing' is not defined

In [128]:
import time
layout_path = "1.png"  # Replace with your image path
layout, layout_denoised, layout_bin, th = pre_processing(layout_path)
blobs_layout = connected_components(layout_bin)

SEM_path = layout_path  # Replace with your image path
SEM, SEM_denoised, SEM_bin, th = pre_processing(SEM_path)
blobs_SEM = connected_components(SEM_bin)

blobs_layout = connected_components(layout_bin)
blobs_SEM = connected_components(SEM_bin)
# Perform your operations here

# Record the end time

start_time = time.time()
j = 0
k=0
for blob_layout, blob_SEM in zip(blobs_layout, blobs_SEM):
    j = j+1
    filled_image = binary_fill_holes(blob_layout.image)
    filled_image = filled_image.astype(np.uint8)
#     contour_image = np.zeros_like(filled_image)
    
    contour = detect_contour(filled_image)
#     print(len(contour))
    
    if len(contour) < 2:
        k = k+1
        continue
    
    gx,gy = sample_polygon_uniformly(contour,50)
    contour_sampled = np.column_stack((gx, gy)).astype(np.int32)
#     cv2.drawContours(contour_image, [contour], 0, (255), thickness=1)
#     f_descriptors_layout = fourier_descriptors(contour_sampled)
    
    zernike_layout = mahotas.features.zernike_moments(filled_image, cv2.minEnclosingCircle(contour_sampled)[1], degree=8)
    
    filled_image = binary_fill_holes(blob_SEM.image)
    filled_image = filled_image.astype(np.uint8)
#     contour_image = np.zeros_like(filled_image)
    
    contour = detect_contour(filled_image)
    gx,gy = sample_polygon_uniformly(contour,50)
    contour_sampled = np.column_stack((gx, gy)).astype(np.int32)
#     cv2.drawContours(contour_image, [contour], 0, (255), thickness=1)
#     f_descriptors_SEM = fourier_descriptors(contour_sampled)
    
    zernike_SEM = mahotas.features.zernike_moments(filled_image, cv2.minEnclosingCircle(contour_sampled)[1], degree=8)
    
#     f_similarity = calculate_similarity(f_descriptors_layout, f_descriptors_SEM)
    z_similarity = calculate_similarity(zernike_layout, zernike_SEM)
#     print(zernike_SEM)
#     print(calculate_similarity(f_descriptors_layout, f_descriptors_SEM))
#     print(calculate_similarity(zernike_layout, zernike_SEM))
    
end_time = time.time()

# Calculate the runtime
runtime_z = end_time - start_time
print("cell number = ",j)
print("runtime_z = ",runtime)

start_time = time.time()
j = 0
for blob_layout, blob_SEM in zip(blobs_layout, blobs_SEM):
    j = j+1
    filled_image = binary_fill_holes(blob_layout.image)
    filled_image = filled_image.astype(np.uint8)
#     contour_image = np.zeros_like(filled_image)
    
    contour = detect_contour(filled_image)
    if len(contour) < 2:
        continue
    gx,gy = sample_polygon_uniformly(contour,50)
    contour_sampled = np.column_stack((gx, gy)).astype(np.int32)
#     cv2.drawContours(contour_image, [contour], 0, (255), thickness=1)
    f_descriptors_layout = fourier_descriptors(contour_sampled)
    
#     zernike_layout = mahotas.features.zernike_moments(filled_image, cv2.minEnclosingCircle(contour_sampled)[1], degree=8)
    
    filled_image = binary_fill_holes(blob_SEM.image)
    filled_image = filled_image.astype(np.uint8)
#     contour_image = np.zeros_like(filled_image)
    
    contour = detect_contour(filled_image)
    gx,gy = sample_polygon_uniformly(contour,50)
    contour_sampled = np.column_stack((gx, gy)).astype(np.int32)
#     cv2.drawContours(contour_image, [contour], 0, (255), thickness=1)
    f_descriptors_SEM = fourier_descriptors(contour_sampled)
    
#     zernike_SEM = mahotas.features.zernike_moments(filled_image, cv2.minEnclosingCircle(contour_sampled)[1], degree=8)
    
    f_similarity = calculate_similarity(f_descriptors_layout, f_descriptors_SEM)
#     z_similarity = calculate_similarity(zernike_layout, zernike_SEM)
#     print(zernike_SEM)
#     print(calculate_similarity(f_descriptors_layout, f_descriptors_SEM))
#     print(calculate_similarity(zernike_layout, zernike_SEM))
    
end_time = time.time()

# Calculate the runtime
runtime_f = end_time - start_time
print("fourier = ",runtime)

start_time = time.time()
j = 0
for blob_layout, blob_SEM in zip(blobs_layout, blobs_SEM):
    j = j+1
    filled_image = binary_fill_holes(blob_layout.image)
    filled_image = filled_image.astype(np.uint8)
#     contour_image = np.zeros_like(filled_image)
    
    contour = detect_contour(filled_image)
    if len(contour) < 2:
        continue
    gx,gy = sample_polygon_uniformly(contour,50)
    contour_sampled = np.column_stack((gx, gy)).astype(np.int32)
#     cv2.drawContours(contour_image, [contour], 0, (255), thickness=1)
    f_descriptors_layout = fourier_descriptors(contour_sampled)
    
    zernike_layout = mahotas.features.zernike_moments(filled_image, cv2.minEnclosingCircle(contour_sampled)[1], degree=8)
    
    filled_image = binary_fill_holes(blob_SEM.image)
    filled_image = filled_image.astype(np.uint8)
#     contour_image = np.zeros_like(filled_image)
    
    contour = detect_contour(filled_image)
    gx,gy = sample_polygon_uniformly(contour,50)
    contour_sampled = np.column_stack((gx, gy)).astype(np.int32)
#     cv2.drawContours(contour_image, [contour], 0, (255), thickness=1)
    f_descriptors_SEM = fourier_descriptors(contour_sampled)
    
    zernike_SEM = mahotas.features.zernike_moments(filled_image, cv2.minEnclosingCircle(contour_sampled)[1], degree=8)
    
    f_similarity = calculate_similarity(f_descriptors_layout, f_descriptors_SEM)
    z_similarity = calculate_similarity(zernike_layout, zernike_SEM)
#     print(zernike_SEM)
#     print(calculate_similarity(f_descriptors_layout, f_descriptors_SEM))
#     print(calculate_similarity(zernike_layout, zernike_SEM))
    
end_time = time.time()

# Calculate the runtime
runtime_f_z = end_time - start_time
print("fourier+zernike = ",runtime)

from sklearn.cluster import KMeans
def calculate_iou(box_lout, box_sem):
    box_lout = list(box_lout)
    box_sem = list(box_sem)

    x_left = max(box_lout[0], box_sem[0])
    y_top = max(box_lout[1], box_sem[1])
    x_right = min(box_lout[2], box_sem[2])
    y_bottom = min(box_lout[3], box_sem[3])
    
    if (x_right < x_left) or (y_bottom < y_top):
        return 0.0
    
    intersection_area = (x_right - x_left) * (y_bottom - y_top)

    # compute the area of both AABBs
    bb_lout_area = (box_lout[2] - box_lout[0]) * (box_lout[3] - box_lout[1])
    bb_sem_area = (box_sem[2] - box_sem[0]) * (box_sem[3] - box_sem[1])

    iou = intersection_area / float(bb_lout_area + bb_sem_area - intersection_area)
    return iou

def bboxed(bin_img):
    blobs = connected_components(bin_img)
    bboxes, _ , _, _, _ = regions(blobs, bin_img, partial_check=False)
    return bboxes, _

def finding_neighbor(bboxes_sem, bboxes_lout):
    neighbors = np.full(len(bboxes_sem),-1)
    centroid = []
    
    centroid_sem = [[(bbox[0]+bbox[2])/2, (bbox[1]+bbox[3])/2] for bbox in bboxes_sem]
    centroid.extend(centroid_sem)
    
    centroid_lout = [[(bbox[0]+bbox[2])/2, (bbox[1]+bbox[3])/2] for bbox in bboxes_lout]
    centroid.extend(centroid_lout)

    centroid = np.asarray(centroid)

    n_cluster = int(len(centroid)/2)
#     print('n_cluster: ',n_cluster)
    kmeans = KMeans(n_clusters=n_cluster, random_state=0).fit(centroid)
    labels = np.array(kmeans.labels_)
#     print('labels: ',labels)

    for i in range(n_cluster):
        indx = np.where(labels==i)
#         print(indx[0])
        index_1, index_2 = int(indx[0][0]), int(indx[0][1])

        neighbors[index_1] = index_2-len(neighbors)    
    
    return neighbors, centroid_sem, centroid_lout



bboxes_sem, _ = bboxed(SEM_bin)
bboxes_lout, _ = bboxed(layout_bin)

start_time = time.time()
neighbors, centroid_sem, centroid_lout = finding_neighbor(bboxes_sem, bboxes_lout)


for i in range(len(neighbors)):
    iou = calculate_iou(bboxes_lout[neighbors[i]], bboxes_sem[i])
#     print('iou between layout comp ',neighbors[i], ' and sem comp ',i, ': ',iou)

end_time = time.time()

# Calculate the runtime
runtime_nidish = end_time - start_time
# for blob_layout, blob_SEM in zip(blobs_layout, blobs_SEM):
#     iou = calculate_iou(blob_layout.bbox, blob_SEM.bbox)
#     print(iou)
print("nidish = ",runtime)






from openpyxl import load_workbook

def get_last_row(sheet):
    for row in range(sheet.max_row, 0, -1):
        if any(sheet.cell(row=row, column=col).value for col in range(1, sheet.max_column + 1)):
            return row
    return 0

# Load the Excel file
wb = load_workbook('runtime.xlsx')

# Select the active worksheet
ws = wb.active

# Get the last row with data
last_row = get_last_row(ws)

# Append values to specific columns starting from the next row
data_to_append = [
    (j, k, runtime_nidish,runtime_f,runtime_z,runtime_f_z,)  # Row 1
]

for row_index, row_data in enumerate(data_to_append, start=1):
    for col_index, value in enumerate(row_data, start=1):
        ws.cell(row=last_row + row_index, column=col_index, value=value)

# Save the changes to the Excel file
wb.save('runtime.xlsx')

img shape:  (936, 975)
 OT threshold:  118.0
img shape:  (936, 975)
 OT threshold:  118.0
cell number =  1062
runtime_z =  0.24596858024597168
fourier =  0.24596858024597168
fourier+zernike =  0.24596858024597168


/home/sekiro/.local/lib/python3.8/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


nidish =  0.24596858024597168


In [101]:
import time


blobs_layout = connected_components(layout_bin)
blobs_SEM = connected_components(SEM_bin)
# Perform your operations here

# Record the end time

start_time = time.time()
j = 0
for blob_layout, blob_SEM in zip(blobs_layout, blobs_SEM):
    j = j+1
    filled_image = binary_fill_holes(blob_layout.image)
    filled_image = filled_image.astype(np.uint8)
#     contour_image = np.zeros_like(filled_image)
    
    contour = detect_contour(filled_image)
    gx,gy = sample_polygon_uniformly(contour,50)
    contour_sampled = np.column_stack((gx, gy)).astype(np.int32)
#     cv2.drawContours(contour_image, [contour], 0, (255), thickness=1)
#     f_descriptors_layout = fourier_descriptors(contour_sampled)
    
    zernike_layout = mahotas.features.zernike_moments(filled_image, cv2.minEnclosingCircle(contour_sampled)[1], degree=8)
    
    filled_image = binary_fill_holes(blob_SEM.image)
    filled_image = filled_image.astype(np.uint8)
#     contour_image = np.zeros_like(filled_image)
    
    contour = detect_contour(filled_image)
    gx,gy = sample_polygon_uniformly(contour,50)
    contour_sampled = np.column_stack((gx, gy)).astype(np.int32)
#     cv2.drawContours(contour_image, [contour], 0, (255), thickness=1)
#     f_descriptors_SEM = fourier_descriptors(contour_sampled)
    
    zernike_SEM = mahotas.features.zernike_moments(filled_image, cv2.minEnclosingCircle(contour_sampled)[1], degree=8)
    
#     f_similarity = calculate_similarity(f_descriptors_layout, f_descriptors_SEM)
    z_similarity = calculate_similarity(zernike_layout, zernike_SEM)
#     print(zernike_SEM)
#     print(calculate_similarity(f_descriptors_layout, f_descriptors_SEM))
#     print(calculate_similarity(zernike_layout, zernike_SEM))
    
end_time = time.time()

# Calculate the runtime
runtime = end_time - start_time
print("cell number = ",j)
print("zernike = ",runtime)


IndexError: index 1 is out of bounds for axis 0 with size 1

In [35]:
start_time = time.time()
j = 0
for blob_layout, blob_SEM in zip(blobs_layout, blobs_SEM):
    j = j+1
    filled_image = binary_fill_holes(blob_layout.image)
    filled_image = filled_image.astype(np.uint8)
#     contour_image = np.zeros_like(filled_image)
    
    contour = detect_contour(filled_image)
    gx,gy = sample_polygon_uniformly(contour,50)
    contour_sampled = np.column_stack((gx, gy)).astype(np.int32)
#     cv2.drawContours(contour_image, [contour], 0, (255), thickness=1)
    f_descriptors_layout = fourier_descriptors(contour_sampled)
    
#     zernike_layout = mahotas.features.zernike_moments(filled_image, cv2.minEnclosingCircle(contour_sampled)[1], degree=8)
    
    filled_image = binary_fill_holes(blob_SEM.image)
    filled_image = filled_image.astype(np.uint8)
#     contour_image = np.zeros_like(filled_image)
    
    contour = detect_contour(filled_image)
    gx,gy = sample_polygon_uniformly(contour,50)
    contour_sampled = np.column_stack((gx, gy)).astype(np.int32)
#     cv2.drawContours(contour_image, [contour], 0, (255), thickness=1)
    f_descriptors_SEM = fourier_descriptors(contour_sampled)
    
#     zernike_SEM = mahotas.features.zernike_moments(filled_image, cv2.minEnclosingCircle(contour_sampled)[1], degree=8)
    
    f_similarity = calculate_similarity(f_descriptors_layout, f_descriptors_SEM)
#     z_similarity = calculate_similarity(zernike_layout, zernike_SEM)
#     print(zernike_SEM)
#     print(calculate_similarity(f_descriptors_layout, f_descriptors_SEM))
#     print(calculate_similarity(zernike_layout, zernike_SEM))
    
end_time = time.time()

# Calculate the runtime
runtime = end_time - start_time
print("fourier = ",runtime)

0.19565725326538086


In [34]:
start_time = time.time()
j = 0
for blob_layout, blob_SEM in zip(blobs_layout, blobs_SEM):
    j = j+1
    filled_image = binary_fill_holes(blob_layout.image)
    filled_image = filled_image.astype(np.uint8)
#     contour_image = np.zeros_like(filled_image)
    
    contour = detect_contour(filled_image)
    gx,gy = sample_polygon_uniformly(contour,50)
    contour_sampled = np.column_stack((gx, gy)).astype(np.int32)
#     cv2.drawContours(contour_image, [contour], 0, (255), thickness=1)
    f_descriptors_layout = fourier_descriptors(contour_sampled)
    
    zernike_layout = mahotas.features.zernike_moments(filled_image, cv2.minEnclosingCircle(contour_sampled)[1], degree=8)
    
    filled_image = binary_fill_holes(blob_SEM.image)
    filled_image = filled_image.astype(np.uint8)
#     contour_image = np.zeros_like(filled_image)
    
    contour = detect_contour(filled_image)
    gx,gy = sample_polygon_uniformly(contour,50)
    contour_sampled = np.column_stack((gx, gy)).astype(np.int32)
#     cv2.drawContours(contour_image, [contour], 0, (255), thickness=1)
    f_descriptors_SEM = fourier_descriptors(contour_sampled)
    
    zernike_SEM = mahotas.features.zernike_moments(filled_image, cv2.minEnclosingCircle(contour_sampled)[1], degree=8)
    
    f_similarity = calculate_similarity(f_descriptors_layout, f_descriptors_SEM)
    z_similarity = calculate_similarity(zernike_layout, zernike_SEM)
#     print(zernike_SEM)
#     print(calculate_similarity(f_descriptors_layout, f_descriptors_SEM))
#     print(calculate_similarity(zernike_layout, zernike_SEM))
    
end_time = time.time()

# Calculate the runtime
runtime = end_time - start_time
print("fourier+zernike = ",runtime)

0.3226509094238281


In [25]:
from sklearn.cluster import KMeans
def calculate_iou(box_lout, box_sem):
    box_lout = list(box_lout)
    box_sem = list(box_sem)

    x_left = max(box_lout[0], box_sem[0])
    y_top = max(box_lout[1], box_sem[1])
    x_right = min(box_lout[2], box_sem[2])
    y_bottom = min(box_lout[3], box_sem[3])
    
    if (x_right < x_left) or (y_bottom < y_top):
        return 0.0
    
    intersection_area = (x_right - x_left) * (y_bottom - y_top)

    # compute the area of both AABBs
    bb_lout_area = (box_lout[2] - box_lout[0]) * (box_lout[3] - box_lout[1])
    bb_sem_area = (box_sem[2] - box_sem[0]) * (box_sem[3] - box_sem[1])

    iou = intersection_area / float(bb_lout_area + bb_sem_area - intersection_area)
    return iou

def bboxed(bin_img):
    blobs = connected_components(bin_img)
    bboxes, _ , _, _, _ = regions(blobs, bin_img, partial_check=False)
    return bboxes, _

def finding_neighbor(bboxes_sem, bboxes_lout):
    neighbors = np.full(len(bboxes_sem),-1)
    centroid = []
    
    centroid_sem = [[(bbox[0]+bbox[2])/2, (bbox[1]+bbox[3])/2] for bbox in bboxes_sem]
    centroid.extend(centroid_sem)
    
    centroid_lout = [[(bbox[0]+bbox[2])/2, (bbox[1]+bbox[3])/2] for bbox in bboxes_lout]
    centroid.extend(centroid_lout)

    centroid = np.asarray(centroid)

    n_cluster = int(len(centroid)/2)
#     print('n_cluster: ',n_cluster)
    kmeans = KMeans(n_clusters=n_cluster, random_state=0).fit(centroid)
    labels = np.array(kmeans.labels_)
#     print('labels: ',labels)

    for i in range(n_cluster):
        indx = np.where(labels==i)
#         print(indx[0])
        index_1, index_2 = int(indx[0][0]), int(indx[0][1])

        neighbors[index_1] = index_2-len(neighbors)    
    
    return neighbors, centroid_sem, centroid_lout



bboxes_sem, _ = bboxed(SEM_bin)
bboxes_lout, _ = bboxed(layout_bin)

start_time = time.time()
neighbors, centroid_sem, centroid_lout = finding_neighbor(bboxes_sem, bboxes_lout)


for i in range(len(neighbors)):
    iou = calculate_iou(bboxes_lout[neighbors[i]], bboxes_sem[i])
#     print('iou between layout comp ',neighbors[i], ' and sem comp ',i, ': ',iou)

end_time = time.time()

# Calculate the runtime
runtime = end_time - start_time
# for blob_layout, blob_SEM in zip(blobs_layout, blobs_SEM):
#     iou = calculate_iou(blob_layout.bbox, blob_SEM.bbox)
#     print(iou)
print("nidish = ",runtime)

/home/sekiro/.local/lib/python3.8/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


0.5012261867523193


In [ ]:
image_path = "28.jpg"  # Replace with your image path
img = read_input(image_path)
img, img_denoised, img_bin, th = pre_processing(image_path)
# cv2.imshow('binarized image', img_bin)
# cv2.waitKey(0) 
  
# # closing all open windows 
# cv2.destroyAllWindows() 

# print('img shape: ',img_denoised.shape)
blobs = connected_components(img_bin)

j = 0
for i in blobs:
#     (min_row, min_col, max_row, max_col) = i.bbox
#     org = img_bin[min_row:max_row, min_col:max_col]
    
    filled_image = binary_fill_holes(i.image)
    filled_image = filled_image.astype(np.uint8)
    contour_image = np.zeros_like(filled_image)
    
    contours = detect_contour(filled_image)
    gx,gy = sample_polygon_uniformly(contours,50)
    contour = np.column_stack((gx, gy)).astype(np.int32)
    cv2.drawContours(contour_image, [contour], 0, (255), thickness=1)

# # Display the contour image
#     plt.imshow(contour_image, cmap='gray')
#     plt.title('Detected Contour')
#     plt.axis('off')
#     plt.show()
    
    fig, axes = plt.subplots(1, 2, figsize=(10, 5))  # 1 row, 2 columns

    # Display the images in subplots
    axes[0].imshow(i.image, cmap='gray', vmin=0, vmax=1)
    axes[0].set_title('Image 1')

    axes[1].imshow(contour_image, cmap='gray')
    axes[1].set_title('Image 2')

# Adjust spacing between subplots
    plt.tight_layout()

# Show the plot
    plt.show()
    print(j)
    j = j + 1
    


In [116]:
from openpyxl import Workbook

# Create a new Workbook object
wb = Workbook()

# Select the active worksheet
ws = wb.active

# Write data to specific cells
# ws['A1'] = 'Hello'
# ws['B1'] = 'World!'

# Save the workbook with a specified filename
wb.save('new_excel_file.xlsx')